# EDA For Offensive Language Classification
for now im trying to only deal with englishm trying to replicate the english offensive classification model

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import nltk
from collections import Counter
nltk.download('stopwords')

from nltk.corpus import stopwords
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/balamurugan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Lets import the tamil dataset 

In [29]:
df = pd.read_csv('Dataset/tamil_offensive_full_train.csv')

df["Labels"].unique()

array(['Not_offensive', 'not-Tamil', 'Offensive_Targeted_Insult_Other',
       'Offensive_Targeted_Insult_Group', 'Offensive_Untargetede',
       'Offensive_Targeted_Insult_Individual'], dtype=object)

## Lets try to preprocess the text

In [30]:
def clean(text):
    stw = stopwords.words('english')
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = " ".join([word for word in text.split() if word not in stw])
    
    return text
    
def preprocess(df):
    mapClass = {'Not_offensive':0, 'not-Tamil':1,'Offensive_Targeted_Insult_Other':2,'Offensive_Targeted_Insult_Group':2, 'Offensive_Untargetede':3,'Offensive_Targeted_Insult_Individual':2}
    df['Text'] = df['Text'].apply(clean)
    df = df[df['Text'] != '']
    df["Labels"] = df["Labels"].apply(lambda x : mapClass[x])
    return df
    

In [31]:
df = preprocess(df)
df

/var/folders/ys/19b63m0164d7n0v0p6t_gr980000gn/T/ipykernel_46980/394402801.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Labels"] = df["Labels"].apply(lambda x : mapClass[x])


,Text,Labels
0,movie vara level la erika poguthu,0
1,love ajith kumar vivegam movie inki mjy bht achi lgi,1
2,padam nalla comedy padama irukum polaye,0
3,karthick subburaj anne intha padam vetri adaya unagalukku ennudaya valthukkal,0
5,ippo intha trailer ah parkuravana oru like podunga,0
...,...,...
35133,inda pathi like thala fans um thandrukanga da mooditu poo,2
35134,trending number idhukku nammalam karanamnu sollumbodhe oru thani gethu dhan ya kshatrian da,0
35135,movie script super athuvum hip hop tamizha music vera,0
35136,k likes k likes,0


In [32]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Dropout, Layer
from tensorflow.keras.layers import Embedding, Input, GlobalAveragePooling1D, Dense
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, Model
import numpy as np
import warnings


In [33]:
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential(
            [Dense(ff_dim, activation="relu"), 
             Dense(embed_dim),]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=True):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [34]:
class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [53]:
X = df["Text"]
y = df["Labels"]

In [54]:
vocab_size = 20000  
maxlen = 200  

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


text = X.tolist()  
def tokenizer(text)
    tokenizer = Tokenizer(num_words=vocab_size)

    tokenizer.fit_on_texts(text)
    sequence = tokenizer.texts_to_sequences(text)
    X = pad_sequences(sequence, maxlen=maxlen)
    return X
X

array([[   0,    0,    0, ...,    6, 2340,  411],
       [   0,    0,    0, ..., 6212, 1802, 8321],
       [   0,    0,    0, ...,  279,   90, 1093],
       ...,
       [   0,    0,    0, ...,  942,   55,    5],
       [   0,    0,    0, ...,   28,   20,   28],
       [   0,    0,    0, ..., 3047,  435,  743]], dtype=int32)

In [55]:
from sklearn.model_selection import train_test_split

x_train_pad, x_val_pad, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(len(x_train_pad), "Training sequences")
print(len(x_val_pad), "Validation sequences")


23916 Training sequences
5980 Validation sequences


In [57]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train_pad, maxlen=maxlen)
x_val = tf.keras.preprocessing.sequence.pad_sequences(x_val_pad, maxlen=maxlen)

In [73]:
embed_dim = 64   
num_heads = 4    
ff_dim = 128     

inputs = Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.3)(x)
x = Dense(64, activation="relu")(x)
x = Dropout(0.3)(x)
outputs = Dense(4, activation="softmax")(x)

model = Model(inputs=inputs, outputs=outputs)

In [74]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

history = model.fit(x_train, y_train, batch_size=128, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 73s 383ms/step - accuracy: 0.6916 - loss: 0.9685 - val_accuracy: 0.7048 - val_loss: 0.9023
Epoch 2/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 77s 413ms/step - accuracy: 0.7093 - loss: 0.9190 - val_accuracy: 0.7048 - val_loss: 0.8604
Epoch 3/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 86s 462ms/step - accuracy: 0.7115 - loss: 0.7592 - val_accuracy: 0.7423 - val_loss: 0.6734
Epoch 4/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 75s 400ms/step - accuracy: 0.7841 - loss: 0.5567 - val_accuracy: 0.7433 - val_loss: 0.6564
Epoch 5/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 64s 344ms/step - accuracy: 0.8246 - loss: 0.4672 - val_accuracy: 0.7497 - val_loss: 0.6941
Epoch 6/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 65s 347ms/step - accuracy: 0.8501 - loss: 0.3884 - val_accuracy: 0.7212 - val_loss: 0.7453
Epoch 7/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 69s 367ms/step - accuracy: 0.8737 - loss: 0.3343 - val_accuracy: 0.7410 - val_loss: 0.8858
Epoch 8/10
187/187 ━━━━━━━━━━━━━━━━━━━━ 78s 415ms/step - accuracy: 0.9051 - loss: 0